# DETECTANDO NORMALIDAD Y AJUSTE DE DISTRIBUCION
IMPORTANDO LIBRERIAS

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.graphics.gofplots import qqplot
from scipy.stats import shapiro
from scipy.stats import normaltest
import warnings
import scipy.stats as st
import statsmodels as sm
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.axes





LEYENDO DATASET

In [ ]:
df=pd.read_csv('../input/diabetes-diagnosis/diabetesdiagnosis.csv', sep=',')


DETECTANDO NORMALIDAD APLICANDO CUANTILES TEÓRICOS

In [ ]:
def qqplotJona(datos):
    headers=list(datos.columns.values)
    for i in range(0,len(headers)):   
        try:
            with warnings.catch_warnings():
                warnings.filterwarnings('ignore')
                print(headers[i])        
                qqplot(df[headers[i]] , line='s')
                plt.show()
        except Exception:
            pass
            

In [ ]:
qqplotJona(df)

DETECTANDO NORMALIDAD APLICANDO SHAPIRO-WILK

In [ ]:
def shapiroJona(datos):
    headers=list(datos.columns.values)
    for i in range(0,len(headers)):    
        print(headers[i])        
        stat, p = shapiro(df[headers[i]].to_numpy())
        print('Estadisticos=%.3f, p=%.3f' % (stat, p))            
        print('Estadisticos=',stat)            
        print('p=',p)            
        alpha = 0.05
        if p > alpha:
           print('La muestra parece Gaussiana o Normal (no se rechaza la hipótesis nula H0)')
        else:
           print('La muestra no parece Gaussiana o Normal(se rechaza la hipótesis nula H0)')


In [ ]:
shapiroJona(df)

DETECTANDO NORMALIDAD APLICANDO K^2 de D’Agostino

In [ ]:
def dAgostinoJona(datos):
    headers=list(datos.columns.values)
    for i in range(0,len(headers)):    
        print(headers[i])        
        stat, p = normaltest(df[headers[i]].to_numpy())
        print('Estadisticos=%.3f, p=%.3f' % (stat, p))            
        print('Estadisticos=',stat)            
        print('p=',p)            
        alpha = 0.05
        if p > alpha:
           print('La muestra parece Gaussiana o Normal (no se rechaza la hipótesis nula H0)')
        else:
           print('La muestra no parece Gaussiana o Normal(se rechaza la hipótesis nula H0)')

In [ ]:
dAgostinoJona(df)

AJUSTE DE DISTRIBUCIÓN A TRAVÉS DE Kolmogorov-Smirnov

In [ ]:
def getBestDistributionJona(data):
    dist_names = [
        "norm","invgauss","johnsonsu","cauchy","vonmises_line","cauchy","vonmises_line","exponnorm","hypsecant"
        # "alpha","anglit","arcsine","beta","betaprime","bradford","burr","cauchy","chi","chi2","cosine","dgamma","dweibull","erlang","expon","exponnorm","exponweib","exponpow","f","fatiguelife","fisk","foldcauchy","foldnorm","frechet_r","frechet_l","genlogistic","genpareto","gennorm","genexpon","genextreme","gausshyper","gamma","gengamma","genhalflogistic","gilbrat","gompertz","gumbel_r","gumbel_l","halfcauchy","halflogistic","halfnorm","halfgennorm","hypsecant","invgamma","invgauss","invweibull","johnsonsb","johnsonsu","ksone","kstwobign","laplace","levy","levy_l","levy_stable","logistic","loggamma","loglaplace","lognorm","lomax","maxwell","mielke","nakagami","ncx2","ncf","nct","norm","pareto","pearson3","powerlaw","powerlognorm","powernorm","rdist","reciprocal","rayleigh","rice","recipinvgauss","semicircular","t","triang","truncexpon","truncnorm","tukeylambda","uniform","vonmises","vonmises_line","wald","weibull_min","weibull_max","wrapcauchy"
    ]
    dist_results = []
    params = {}
    for dist_name in dist_names:
      # Try to fit the distribution
        try:
            # Ignore warnings from data that can't be fit
            with warnings.catch_warnings():
                warnings.filterwarnings('ignore')
                dist = getattr(st, dist_name)
                param = dist.fit(data)

                params[dist_name] = param
                # Applying the Kolmogorov-Smirnov test
                D, p = st.kstest(data, dist_name, args=param)
                print("p value for "+dist_name+" = "+str(p))
                dist_results.append((dist_name, p))
        except Exception:
            pass
    # select the best fitted distribution
    best_dist, best_p = (max(dist_results, key=lambda item: item[1]))
    # store the name of the best fit and its p value

    print("Best fitting distribution: "+str(best_dist))
    print("Best p value: "+ str(best_p))
    print("Parameters for the best fit: "+ str(params[best_dist]))

    return best_dist, best_p, params[best_dist]

In [ ]:
#getBestDistributionJona(df['Pregnancies']) ejemplo de uso
def KolmogorovSmirnov(datos):    
    headers=list(datos.columns.values)
    for i in range(0,len(headers)):    
        print(headers[i]) 
        getBestDistributionJona(df[headers[i]])

In [ ]:
KolmogorovSmirnov(df)

AJUSTE DE DISTRIBUCIÓN USANDO SUMA DE CUADRADOS DEL ERROR

In [ ]:
def best_fit_distribution(data, bins=200, ax=None):
    """Model data by finding best fit distribution to data"""
    # Get histogram of original data
    y, x = np.histogram(data, bins=bins, density=True)
    x = (x + np.roll(x, -1))[:-1] / 2.0

    # Distributions to check
    DISTRIBUTIONS = [        
        st.norm,st.invgauss,st.johnsonsu,st.cauchy,st.vonmises_line,st.cauchy,st.vonmises_line,st.exponnorm,st.hypsecant
        #st.alpha,st.anglit,st.arcsine,st.beta,st.betaprime,st.bradford,st.burr,st.cauchy,st.chi,st.chi2,st.cosine,st.dgamma,st.dweibull,st.erlang,st.expon,st.exponnorm,st.exponweib,st.exponpow,st.f,st.fatiguelife,st.fisk,st.foldcauchy,st.foldnorm,st.frechet_r,st.frechet_l,st.genlogistic,st.genpareto,st.gennorm,st.genexpon,st.genextreme,st.gausshyper,st.gamma,st.gengamma,st.genhalflogistic,st.gilbrat,st.gompertz,st.gumbel_r,st.gumbel_l,st.halfcauchy,st.halflogistic,st.halfnorm,st.halfgennorm,st.hypsecant,st.invgamma,st.invgauss,st.invweibull,st.johnsonsb,st.johnsonsu,st.ksone,st.kstwobign,st.laplace,st.levy,st.levy_l,st.levy_stable,st.logistic,st.loggamma,st.loglaplace,st.lognorm,st.lomax,st.maxwell,st.mielke,st.nakagami,st.ncx2,st.ncf,st.nct,st.norm,st.pareto,st.pearson3,st.powerlaw,st.powerlognorm,st.powernorm,st.rdist,st.reciprocal,st.rayleigh,st.rice,st.recipinvgauss,st.semicircular,st.t,st.triang,st.truncexpon,st.truncnorm,st.tukeylambda,st.uniform,st.vonmises,st.vonmises_line,st.wald,st.weibull_min,st.weibull_max,st.wrapcauchy
    ]

    # Best holders
    best_distribution = st.norm
    best_params = (0.0, 1.0)
    best_sse = np.inf

    # Estimate distribution parameters from data
    for distribution in DISTRIBUTIONS:

        # Try to fit the distribution
        try:
            # Ignore warnings from data that can't be fit
            with warnings.catch_warnings():
                warnings.filterwarnings('ignore')

                # fit dist to data
                params = distribution.fit(data)

                # Separate parts of parameters
                arg = params[:-2]
                loc = params[-2]
                scale = params[-1]

                # Calculate fitted PDF and error with fit in distribution
                pdf = distribution.pdf(x, loc=loc, scale=scale, *arg)
                sse = np.sum(np.power(y - pdf, 2.0))

                # if axis pass in add to plot
                try:
                    if ax:
                        pd.Series(pdf, x).plot(ax=ax)
                    end
                except Exception:
                    pass

                # identify if this distribution is better
                if best_sse > sse > 0:
                    best_distribution = distribution
                    best_params = params
                    best_sse = sse

        except Exception:
            pass

    return (best_distribution.name, best_params)

In [ ]:
def make_pdf(dist, params, size=10000):
    """Generate distributions's Probability Distribution Function """

    # Separate parts of parameters
    arg = params[:-2]
    loc = params[-2]
    scale = params[-1]

    # Get sane start and end points of distribution
    start = dist.ppf(0.01, *arg, loc=loc, scale=scale) if arg else dist.ppf(0.01, loc=loc, scale=scale)
    end = dist.ppf(0.99, *arg, loc=loc, scale=scale) if arg else dist.ppf(0.99, loc=loc, scale=scale)

    # Build PDF and turn into pandas Series
    x = np.linspace(start, end, size)
    y = dist.pdf(x, loc=loc, scale=scale, *arg)
    pdf = pd.Series(y, x)

    return pdf

In [ ]:
def ajustarDistribucionSSE(datos):
    headers=list(datos.columns.values)
    for i in range(0,len(headers)):   
        try: 
            with warnings.catch_warnings():
                warnings.filterwarnings('ignore')
                titulo=headers[i]
                print(titulo)
                matplotlib.rcParams['figure.figsize'] = (16.0, 12.0)
                matplotlib.style.use('ggplot')
                data=df[headers[i]]
                plt.figure(figsize=(12,8))
                ax = data.plot(kind='hist', bins=50, density=True, alpha=0.5, color='#aabbcc')
                dataYLim = ax.get_ylim()
                # Find best fit distribution
                best_fit_name, best_fit_params = best_fit_distribution(data, 200, ax)
                best_dist = getattr(st, best_fit_name)
                # Update plots
                ax.set_ylim(dataYLim)
                ax.set_title(titulo)
                ax.set_xlabel(titulo)
                ax.set_ylabel('Frequency')
                # Make PDF with best params
                pdf = make_pdf(best_dist, best_fit_params)
                # Display
                plt.figure(figsize=(12,8))
                ax = pdf.plot(lw=2, label='PDF', legend=True)
                data.plot(kind='hist', bins=50, density=True, alpha=0.5, label='Data', legend=True, ax=ax)
                param_names = (best_dist.shapes + ', loc, scale').split(', ') if best_dist.shapes else ['loc', 'scale']
                param_str = ', '.join(['{}={:0.2f}'.format(k,v) for k,v in zip(param_names, best_fit_params)])
                dist_str = '{}({})'.format(best_fit_name, param_str)
                ax.set_title(titulo+' \n' + dist_str)
                ax.set_xlabel(titulo)
                ax.set_ylabel('Frequency')
        except Exception:
            pass

In [ ]:
ajustarDistribucionSSE(df)

# LINKOGRAFÍA
[https://hub.mybinder.turing.ac.uk/user/relopezbriega-blog-8r7y7cqu/notebooks/content/notebooks/DistStatsPy.ipynb](https://hub.mybinder.turing.ac.uk/user/relopezbriega-blog-8r7y7cqu/notebooks/content/notebooks/DistStatsPy.ipynb)
[https://machinelearningparatodos.com/como-saber-si-una-variable-sigue-una-distribucion-normal-en-python/](https://machinelearningparatodos.com/como-saber-si-una-variable-sigue-una-distribucion-normal-en-python/)
[https://www.cienciadedatos.net/documentos/pystats01-ajuste-distribuciones-python.html](https://www.cienciadedatos.net/documentos/pystats01-ajuste-distribuciones-python.html)
[https://stackoverflow.com/questions/37487830/how-to-find-probability-distribution-and-parameters-for-real-data-python-3](https://stackoverflow.com/questions/37487830/how-to-find-probability-distribution-and-parameters-for-real-data-python-3)
[https://stackoverflow.com/questions/6620471/fitting-empirical-distribution-to-theoretical-ones-with-scipy-python](https://stackoverflow.com/questions/6620471/fitting-empirical-distribution-to-theoretical-ones-with-scipy-python)
[https://stackoverflow.com/questions/6615489/fitting-distributions-goodness-of-fit-p-value-is-it-possible-to-do-this-with/16651524#16651524](https://stackoverflow.com/questions/6615489/fitting-distributions-goodness-of-fit-p-value-is-it-possible-to-do-this-with/16651524#16651524)
[https://glowingpython.blogspot.com/2012/07/distribution-fitting-with-scipy.html](https://glowingpython.blogspot.com/2012/07/distribution-fitting-with-scipy.html)
[https://docs.scipy.org/doc/scipy/reference/stats.html](https://docs.scipy.org/doc/scipy/reference/stats.html)
[https://www.statsmodels.org/devel/datasets/generated/elnino.html#el-nino-sea-surface-temperatures](https://www.statsmodels.org/devel/datasets/generated/elnino.html#el-nino-sea-surface-temperatures)
